In [ ]:
%load_ext autoreload
%autoreload 2

import os,sys
import h5py  
import numpy as np
import math,cmath
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['lines.markersize'] = 12
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['legend.fontsize'] = 24
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['figure.titlesize'] = 30
mpl.rcParams.update({"axes.grid" : True})

mpl.rcParams.update({"axes.grid" : True})
import util as yu

ens='cA211.530.24'
lat=yu.LatticeEnsemble(ens)

In [ ]:
path='/project/s1174/lyan/code/projectData/NST_a/test/data_merge/NST_a.h5_main'
data=yu.load(path)
Ncfg=yu.deepKey(data['2pt'],2).shape[0]
print('Ncfg='+str(Ncfg))

In [ ]:
def get2pt(opa,opb,diags=yu.diags_all):
    if opa.split(';')[-1]=='12':
        opa1=';'.join(opa.split(';')[:-1]+['n,pi+']); opa2=';'.join(opa.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get2pt(opa1,opb,diags)-np.sqrt(1/3)*get2pt(opa2,opb,diags)
    if opb.split(';')[-1]=='12':
        opb1=';'.join(opb.split(';')[:-1]+['n,pi+']); opb2=';'.join(opb.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get2pt(opa,opb1,diags)-np.sqrt(1/3)*get2pt(opa,opb2,diags)
    if yu.getNpars(opa,opb)==(2,1):
        return np.conj(get2pt(opb,opa,diags))
    t=data['2pt'][opa+'_'+opb] 
    res=np.sum([t[diag] for diag in t.keys() if diag in diags],axis=0)
    if opb+'_'+opa in data['2pt'].keys(): # contibution for c.c of T-pi0i
        t=data['2pt'][opb+'_'+opa]; diag='T-pi0f'
        res+= np.conj(t[diag]) if diag in t.keys() and diag in diags else 0
    return res
def get2ptMat(ops,diags=yu.diags_all):
    t=np.transpose([[get2pt(opa,opb,diags) for opb in ops] for opa in ops],[2,3,0,1])
    # t=(t+np.conj(np.transpose(t,[0,1,3,2])))/2
    return t

def get3pt(opa,opb,insert,diags=yu.diags_all):
    if opa.split(';')[-1]=='12':
        opa1=';'.join(opa.split(';')[:-1]+['n,pi+']); opa2=';'.join(opa.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get3pt(opa1,opb,insert,diags)-np.sqrt(1/3)*get3pt(opa2,opb,insert,diags)
    if opb.split(';')[-1]=='12':
        opb1=';'.join(opb.split(';')[:-1]+['n,pi+']); opb2=';'.join(opb.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get3pt(opa,opb1,insert,diags)-np.sqrt(1/3)*get3pt(opa,opb2,insert,diags)
    if yu.getNpars(opa,opb)==(2,1):
        return np.conj(get3pt(opb,opa,insert,diags)[:,::-1])*(yu.gtCj[insert.split('_')[0]])
    t=data['3pt'][opa+'_'+opb][insert]
    return np.sum([t[diag] for diag in t.keys() if diag in diags],axis=0)
def get3ptMat(opas,opbs,insert,diags=yu.diags_all,NpiNpi=False):
    skipNpiNpi=lambda opa,opb: yu.getNpars(opa,opb)==(2,2) and not NpiNpi
    t=np.transpose([[get3pt(opa,opb,insert,diags) if not skipNpiNpi(opa,opb) else np.zeros([Ncfg,int(insert.split('_')[-1])+1])  for opb in opbs] for opa in opas],[2,3,0,1])
    # t=(t+np.conj(np.transpose(t,[0,1,3,2])))/2
    return t

In [ ]:
def run2ptGEVP(pt,ofs,diags=yu.diags_all,title=None,ylim1=None,ylim2=None):
    fig, axs = yu.getFigAxs(1,2,Lrow=4*2,Lcol=5*2)
    title=title if title=='None' else 'pt={}; ofs={}'.format(pt,ofs)
    fig.suptitle(None)
    
    Nop=len(ofs)
    tmin=2; tmax0=15
    tRef=1
    
    for l,color,xshift in zip(['l1'],['r'],[0]):
        ops=yu.getops(pt,l,ofs)
        dat=get2ptMat(ops,diags)
        
        irow=0; icol=0
        def func(dat):
            t=np.mean(dat,axis=0)
            (eVals,eVecs)=yu.GEVP(t,tRef)
            En=np.log(eVals/np.roll(eVals,-1,axis=0))
            return En.T
        mean,err,cov=yu.jackknife(dat,func)
        for ind in range(Nop):
            tmax=[15,11,11][ind]
            tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
            axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=['r','g','b'][ind],fmt=['s','d','o'][ind],label='1')
            axs[irow,icol].set_xticks(range(0,tmax0,1))  
        axs[irow,icol].set_ylim(ylim1)

        tmax=15
        if pt=='0,0,0':
            axs[irow,icol].hlines(y=lat.getaEN(0),xmin=tmin,xmax=tmax,color='red',label='1')
            axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='green',label='1',linestyle='dashed')

            handles, _ = axs[irow,icol].get_legend_handles_labels()
            labels = [r'$N(\vec{0})$',r'$N(\vec{1})\pi(\vec{1})$',r'$E_{N}$',r'$E_{N\pi}$']
            order = [0,1,2,3]
            axs[irow,icol].legend([handles[idx] for idx in order],[labels[idx] for idx in order],ncol=2)
        else:
            axs[irow,icol].hlines(y=lat.getaEN(1),xmin=tmin,xmax=tmax,color='red',label='1')
            axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(0),xmin=tmin,xmax=tmax,color='green',label='1',linestyle='dashed') 
            axs[irow,icol].hlines(y=lat.getaEN(0)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='blue',label='1',linestyle='dashdot') 
            # axs[irow,icol].hlines(y=lat.getaEN(2)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='orange') 
            # axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(2),xmin=tmin,xmax=tmax,color='orange') 

            handles, _ = axs[irow,icol].get_legend_handles_labels()
            labels = [r'$N(\vec{1})$',r'$N(\vec{1})\pi(\vec{0})$',r'$N(\vec{0})\pi(\vec{1})$',r'$E_{N}$',r'$E_{N\pi;1}$',r'$E_{N\pi;2}$']
            order = [0,1,2,3,4,5]
            axs[irow,icol].legend([handles[idx] for idx in order],[labels[idx] for idx in order],ncol=2)

        axs[irow,icol].set_xlabel(r'$t_{sink}$')
        axs[irow,icol].set_ylabel(r'effective energies')
        
        irow=0; icol=1
        for j in range(1):
            def func(dat):
                t=np.mean(dat,axis=0)
                (eVals,eVecs)=yu.GEVP(t,tRef)
                t=eVecs[:,:,j].T
                t=t/t[0,:]
                return np.abs(t)
            mean,err,cov=yu.jackknife(dat,func)
            for ind in range(Nop):
                tmax=tmax0
                tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
                axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color={0:(1,0,0),1:(1,0.3,0.3),2:(1,0.5,0.5)}[ind],fmt=['s','<','>'][ind],label='1'.format(j+1))
                axs[irow,icol].set_xticks(range(0,tmax0,1))  
            axs[irow,icol].set_yscale('log')
            axs[irow,icol].set_ylim(ylim2)
            axs[irow,icol].set_yticks([1,1e-1,1e-2,1e-3])
            
            if pt=='0,0,0':
                handles, _ = axs[irow,icol].get_legend_handles_labels()
                labels = [r'$v^N_{N}\equiv1$',r'$v^N_{N\pi}$']
                order = [0,1]
                axs[irow,icol].legend([handles[idx] for idx in order],[labels[idx] for idx in order])
            else:
                handles, _ = axs[irow,icol].get_legend_handles_labels()
                labels = [r'$v^N_{N}\equiv1$',r'$v^N_{N\pi;1}$',r'$v^N_{N\pi;2}$']
                order = [0,1,2]
                axs[irow,icol].legend([handles[idx] for idx in order],[labels[idx] for idx in order])
                
        axs[irow,icol].set_xlabel(r'$t_{sink}$')
        axs[irow,icol].set_ylabel(r'eigenvectors')
  
run2ptGEVP('0,0,0',[('a','p'),('N1pi1,a','12')],ylim1=[0.3,1.1],ylim2=[1e-4,1.5]) 
plt.tight_layout(); plt.savefig('/project/s1174/lyan/code/forDownload/cA24_p000.pdf')

run2ptGEVP('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')],ylim1=[0.3,1.1],ylim2=[1e-4,1.5])  
plt.tight_layout(); plt.savefig('/project/s1174/lyan/code/forDownload/cA24_p001.pdf')

In [ ]:
tfs=[10,12,14]; tRef=1
def get_eVec(eVecs):
    t=eVecs[:,0,0]
    t=eVecs[:,:,0]/t[:,None]
    return np.mean(t[10:14+1,:],axis=0)

def run3pt2ptGEVP(ptofas,ptofbs,insert,prefactor,proj,diags=yu.diags_all,title=None,ylim1=None):
    fig, axs = yu.getFigAxs(1,1,Lrow=4*1.8,Lcol=5*1.8)
    # title=title if title=='None' else 'opa={}; opb={}; insert={}; prefactor={}'.format(opa,opb,insert,prefactor)
    fig.suptitle(title)
    
    prefactor=1 if prefactor=='re' else -1j if prefactor=='im' else prefactor
    proj=np.array([[1,0],[0,1]])/2 if proj=='P0' else np.array([[1,0],[0,-1]])/2 if proj=='Pz' else proj
    
    for case in ['noGEVP','GEVP']:
        pta,ofas=ptofas; ptb,ofbs=ptofbs
        if case=='noGEVP':
            ofas=[ofas[0]]; ofbs=[ofbs[0]]
            
        Nparas=[yu.getNpar(op) for op in yu.getops(pta,'l1',ofas)]
        Nparbs=[yu.getNpar(op) for op in yu.getops(ptb,'l1',ofbs)]
        maska=np.array([[0 if l==r==2 else 1 for l in Nparas] for r in Nparas])
        maskb=np.array([[0 if l==r==2 else 1 for l in Nparbs] for r in Nparbs])
        # maska=np.array([[1 if l==r==2 else 1 for l in Nparas] for r in Nparas])
        # maskb=np.array([[1 if l==r==2 else 1 for l in Nparbs] for r in Nparbs])
            
        needsVEV=pta==ptb and 'N-j' in diags
        
        def t_get(tf):
            t=[[get3ptMat(yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs),insert+'_'+str(tf),diags)[:,1:tf]\
                for lb in ['l1','l2']] for la in ['l1','l2']]
            return np.transpose(t,[2,3,4,5,0,1]) # cfg,tc,opa,opb,la,lb
        
        dat3pt=[t_get(tf) for tf in tfs]
        dat2pt=[get2ptMat(yu.getops(pta,'l1',ofas),yu.diags_all),get2ptMat(yu.getops(pta,'l2',ofas),yu.diags_all),
                get2ptMat(yu.getops(ptb,'l1',ofbs),yu.diags_all),get2ptMat(yu.getops(ptb,'l2',ofbs),yu.diags_all),]
        datVEVj=[data['VEV']['j'][insert]]
        dat=[dat3pt,dat2pt,datVEVj]
        
        AInv=yu.getCoeMat(pta); BInv=yu.getCoeMat(ptb)

        def func(dat):
            t=yu.meanDeep(dat)
            t3pt,t2pt,tVEVj=t
            
            (eVals_al1,eVecs_al1)=yu.GEVP(t2pt[0],tRef)
            (eVals_al2,eVecs_al2)=yu.GEVP(t2pt[1],tRef)
            (eVals_bl1,eVecs_bl1)=yu.GEVP(t2pt[2],tRef)
            (eVals_bl2,eVecs_bl2)=yu.GEVP(t2pt[3],tRef)
            
            eVec_al1=get_eVec(eVecs_al1)
            eVec_al2=get_eVec(eVecs_al2)
            eVec_bl1=get_eVec(eVecs_bl1)
            eVec_bl2=get_eVec(eVecs_bl2)
            
            t2pt_a=(np.einsum('i,tij,j,ij->t',np.conj(eVec_al1),t2pt[0],eVec_al1,maska)+
                    np.einsum('i,tij,j,ij->t',np.conj(eVec_al2),t2pt[1],eVec_al2,maska))/2
            t2pt_b=(np.einsum('i,tij,j,ij->t',np.conj(eVec_bl1),t2pt[2],eVec_bl1,maskb)+
                    np.einsum('i,tij,j,ij->t',np.conj(eVec_bl2),t2pt[3],eVec_bl2,maskb))/2
            
            eVec_a=np.array([eVec_al1,eVec_al2])
            eVec_b=np.array([eVec_bl1,eVec_bl2])
            
            t=[(
                np.einsum('tijlm,ba,li,mj,al,bm->t',t3pt[i],proj,np.conj(eVec_a),eVec_b,AInv,np.conj(BInv)) # ij for ops; l,m for la,lb; ab for diracs
            )/np.sqrt(
                np.real(t2pt_a[tf])*np.real(t2pt_b[tf])*\
                np.real(t2pt_a[1:tf][::-1])/np.real(t2pt_a[1:tf])*\
                np.real(t2pt_b[1:tf])/np.real(t2pt_b[1:tf][::-1])
            ) - (tVEVj if needsVEV else 0) 
            for i,tf in enumerate(tfs)]
            
            t=yu.prefactorDeep(t,prefactor)
            return t
        
        (mean,err,cov)=yu.jackknife(dat,func)

        irow=0; icol=0
        for i_tf,tf in enumerate(tfs):
            tMean=mean[i_tf];tErr=err[i_tf]
            # print(tMean)
            label = {'noGEVP':'no GEVP','GEVP':'GEVP'}[case] if i_tf==0 else '_nolegend_'
            axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,color=['r','g','b'][i_tf],fmt=['s','d','o'][i_tf],mfc='white' if case=='noGEVP' else None, label=label)
        axs[irow,icol].set_xticks(range(-6,7,1))
        axs[irow,icol].set_ylim(ylim1)

        axs[irow,0].legend()
        axs[irow,0].set_xlabel(r'$t_{ins}-t_{sink}/2$')
        axs[irow,0].set_ylabel('3pt/2pt')

run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,0',[('a','p'),('N1pi1,a','12')]),'id_j+',1,'P0',diags=yu.diags_jLoopless,ylim1=[8.4,13])
plt.tight_layout(); plt.savefig('/project/s1174/lyan/code/forDownload/cA24_id+.pdf')
run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,0',[('a','p'),('N1pi1,a','12')]),'g5gz_j-',-1j,'Pz',diags=yu.diags_jLoopless,ylim1=[-1.64,-1.48])
plt.tight_layout(); plt.savefig('/project/s1174/lyan/code/forDownload/cA24_g5gz.pdf')
run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5gt_j-',1,'Pz',diags=yu.diags_jLoopless,ylim1=[-0.2,0.5])
plt.tight_layout(); plt.savefig('/project/s1174/lyan/code/forDownload/cA24_g5gt.pdf')
run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5_j-',1,'Pz',diags=yu.diags_jLoopless,ylim1=[-4.5,-2.5])
plt.tight_layout(); plt.savefig('/project/s1174/lyan/code/forDownload/cA24_g5.pdf')

        
# fig=run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,0',[('a','p'),('N1pi1,a','12')]),'id_j+',1,'P0',diags=yu.diags_jLoopless,ylim1=[8.4,13])
# with PdfPages('/project/s1174/lyan/code/forDownload/cA24_id+.pdf') as pdf:
#     fig.tight_layout()
#     pdf.savefig(fig)
# fig=run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,0',[('a','p'),('N1pi1,a','12')]),'g5gz_j-',-1j,'Pz',diags=yu.diags_jLoopless,ylim1=[-1.64,-1.48])
# with PdfPages('/project/s1174/lyan/code/forDownload/cA24_g5gz.pdf') as pdf:
#     fig.tight_layout()
#     pdf.savefig(fig)
    
# fig=run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5gt_j-',1,'Pz',diags=yu.diags_jLoopless,ylim1=[-0.2,0.5])
# with PdfPages('/project/s1174/lyan/code/forDownload/cA24_g5gt.pdf') as pdf:
#     fig.tight_layout()
#     pdf.savefig(fig)
# run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5_j-',1,'Pz',diags=yu.diags_jLoopless,ylim1=[-4.5,-2.5])
# plt.tight_layout()
# plt.savefig('/project/s1174/lyan/code/forDownload/cA24_g5.pdf')
# with PdfPages('/project/s1174/lyan/code/forDownload/cA24_g5.pdf') as pdf:
#     # fig.tight_layout()
#     pdf.savefig(fig)

In [ ]:
run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5gt_j-',1,'Pz',diags=yu.diags_all,ylim1=[-0.2,0.5])

In [ ]:
run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5_j-',1,'Pz',diags=yu.diags_all,ylim1=[-9,-4])